In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from auctioneer import *
from train_crypto_bot import *

2022-12-06 20:02:14.695771: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
s3 = create_s3()

In [6]:


logging.basicConfig(level = logging.DEBUG)
logger = logging.getLogger(__name__)

STARTING_BALANCE = 100_000
START_DATE = "11-01-2022"
END_DATE = "11-30-2022"



In [44]:
objects = get_all_s3_objects(s3, Bucket=S3_BUCKET, Prefix='TrainedModel')

In [45]:
model, scalers = download_model(s3)

DEBUG:urllib3.connectionpool:Resetting dropped connection: auctioneer1.s3.us-west-1.amazonaws.com
DEBUG:urllib3.connectionpool:https://auctioneer1.s3.us-west-1.amazonaws.com:443 "GET /?list-type=2&max-keys=1000&prefix=TrainedModel&encoding-type=url HTTP/1.1" 200 None
DEBUG:s3transfer.utils:Acquiring 0
DEBUG:s3transfer.tasks:DownloadSubmissionTask(transfer_id=0, {'transfer_future': <s3transfer.futures.TransferFuture object at 0x1924c1f30>}) about to wait for the following futures []
DEBUG:s3transfer.tasks:DownloadSubmissionTask(transfer_id=0, {'transfer_future': <s3transfer.futures.TransferFuture object at 0x1924c1f30>}) done waiting for dependent futures
DEBUG:s3transfer.tasks:Executing task DownloadSubmissionTask(transfer_id=0, {'transfer_future': <s3transfer.futures.TransferFuture object at 0x1924c1f30>}) with kwargs {'client': <botocore.client.S3 object at 0x158c96b00>, 'config': <boto3.s3.transfer.TransferConfig object at 0x1924c1ba0>, 'osutil': <s3transfer.utils.OSUtils object at 

/var/folders/tm/wlbv0c4j0gnb8bk65vggdnkm0000gn/T/tmpejdmyaov/TrainedModel/TrainedModel//fingerprint.pb


DEBUG:urllib3.connectionpool:https://auctioneer1.s3.us-west-1.amazonaws.com:443 "GET /TrainedModel//fingerprint.pb HTTP/1.1" 200 55
DEBUG:s3transfer.tasks:IOWriteTask(transfer_id=0, {'offset': 0}) about to wait for the following futures []
DEBUG:s3transfer.tasks:IOWriteTask(transfer_id=0, {'offset': 0}) done waiting for dependent futures
DEBUG:s3transfer.tasks:Executing task IOWriteTask(transfer_id=0, {'offset': 0}) with kwargs {'fileobj': <s3transfer.utils.DeferredOpenFile object at 0x1924c2200>, 'offset': 0}
DEBUG:s3transfer.tasks:IORenameFileTask(transfer_id=0, {'final_filename': '/var/folders/tm/wlbv0c4j0gnb8bk65vggdnkm0000gn/T/tmpejdmyaov/TrainedModel/TrainedModel//fingerprint.pb'}) about to wait for the following futures []
DEBUG:s3transfer.tasks:IORenameFileTask(transfer_id=0, {'final_filename': '/var/folders/tm/wlbv0c4j0gnb8bk65vggdnkm0000gn/T/tmpejdmyaov/TrainedModel/TrainedModel//fingerprint.pb'}) done waiting for dependent futures
DEBUG:s3transfer.tasks:Executing task IORena

/var/folders/tm/wlbv0c4j0gnb8bk65vggdnkm0000gn/T/tmpejdmyaov/TrainedModel/TrainedModel//keras_metadata.pb


DEBUG:urllib3.connectionpool:https://auctioneer1.s3.us-west-1.amazonaws.com:443 "GET /TrainedModel//keras_metadata.pb HTTP/1.1" 200 26704
DEBUG:s3transfer.tasks:IOWriteTask(transfer_id=0, {'offset': 0}) about to wait for the following futures []
DEBUG:s3transfer.tasks:IOWriteTask(transfer_id=0, {'offset': 0}) done waiting for dependent futures
DEBUG:s3transfer.tasks:Executing task IOWriteTask(transfer_id=0, {'offset': 0}) with kwargs {'fileobj': <s3transfer.utils.DeferredOpenFile object at 0x1924c27a0>, 'offset': 0}
DEBUG:s3transfer.tasks:IORenameFileTask(transfer_id=0, {'final_filename': '/var/folders/tm/wlbv0c4j0gnb8bk65vggdnkm0000gn/T/tmpejdmyaov/TrainedModel/TrainedModel//keras_metadata.pb'}) about to wait for the following futures []
DEBUG:s3transfer.tasks:IORenameFileTask(transfer_id=0, {'final_filename': '/var/folders/tm/wlbv0c4j0gnb8bk65vggdnkm0000gn/T/tmpejdmyaov/TrainedModel/TrainedModel//keras_metadata.pb'}) done waiting for dependent futures
DEBUG:s3transfer.tasks:Executing

...

/var/folders/tm/wlbv0c4j0gnb8bk65vggdnkm0000gn/T/tmpejdmyaov/TrainedModel/TrainedModel//variables/variables.data-00000-of-00001


DEBUG:urllib3.connectionpool:https://auctioneer1.s3.us-west-1.amazonaws.com:443 "GET /TrainedModel//variables/variables.data-00000-of-00001 HTTP/1.1" 200 105823
DEBUG:s3transfer.tasks:IOWriteTask(transfer_id=0, {'offset': 0}) about to wait for the following futures []
DEBUG:s3transfer.tasks:IOWriteTask(transfer_id=0, {'offset': 0}) done waiting for dependent futures
DEBUG:s3transfer.tasks:Executing task IOWriteTask(transfer_id=0, {'offset': 0}) with kwargs {'fileobj': <s3transfer.utils.DeferredOpenFile object at 0x1924c3820>, 'offset': 0}
DEBUG:s3transfer.tasks:IORenameFileTask(transfer_id=0, {'final_filename': '/var/folders/tm/wlbv0c4j0gnb8bk65vggdnkm0000gn/T/tmpejdmyaov/TrainedModel/TrainedModel//variables/variables.data-00000-of-00001'}) about to wait for the following futures []
DEBUG:s3transfer.tasks:IORenameFileTask(transfer_id=0, {'final_filename': '/var/folders/tm/wlbv0c4j0gnb8bk65vggdnkm0000gn/T/tmpejdmyaov/TrainedModel/TrainedModel//variables/variables.data-00000-of-00001'}) 

/var/folders/tm/wlbv0c4j0gnb8bk65vggdnkm0000gn/T/tmpejdmyaov/TrainedModel/TrainedModel//variables/variables.index


DEBUG:urllib3.connectionpool:https://auctioneer1.s3.us-west-1.amazonaws.com:443 "GET /TrainedModel//variables/variables.index HTTP/1.1" 200 2702
DEBUG:s3transfer.tasks:IOWriteTask(transfer_id=0, {'offset': 0}) about to wait for the following futures []
DEBUG:s3transfer.tasks:IOWriteTask(transfer_id=0, {'offset': 0}) done waiting for dependent futures
DEBUG:s3transfer.tasks:Executing task IOWriteTask(transfer_id=0, {'offset': 0}) with kwargs {'fileobj': <s3transfer.utils.DeferredOpenFile object at 0x1924c3160>, 'offset': 0}
DEBUG:s3transfer.tasks:IORenameFileTask(transfer_id=0, {'final_filename': '/var/folders/tm/wlbv0c4j0gnb8bk65vggdnkm0000gn/T/tmpejdmyaov/TrainedModel/TrainedModel//variables/variables.index'}) about to wait for the following futures []
DEBUG:s3transfer.tasks:IORenameFileTask(transfer_id=0, {'final_filename': '/var/folders/tm/wlbv0c4j0gnb8bk65vggdnkm0000gn/T/tmpejdmyaov/TrainedModel/TrainedModel//variables/variables.index'}) done waiting for dependent futures
DEBUG:s3t

OSError: SavedModel file does not exist at: /var/folders/tm/wlbv0c4j0gnb8bk65vggdnkm0000gn/T/tmpejdmyaov/TrainedModel/{saved_model.pbtxt|saved_model.pb}

In [ ]:
tempdir = 't'
download_s3_directory(s3, 'TrainedModel', f"{tempdir}")


In [47]:
model = tf.keras.models.load_model(f"{tempdir}/TrainedModel")

In [48]:
model.predict()

TypeError: Model.predict() missing 1 required positional argument: 'x'